# Observer Pattern

- __Type:__ Behavioral
- __Popularity: ★★★★★__
- __Complexity: ★★☆☆☆__

### Intent:
The Observer pattern defines a one-to-many dependency between objects so that when one object changes state, all its dependents are notified and updated automatically. It allows objects to communicate without being tightly coupled.

### Problem:

You need to be notified when the state of another object changes, but you don't want to continuously check for changes or tightly couple the objects involved. Specifically:

- Multiple objects need to receive updates when a subject object changes state
- The exact set of dependent objects might not be known in advance or could change dynamically
- The system should be flexible enough to add/remove observers without modifying the subject
- The subject shouldn't have to know the concrete classes of its observers

### Solution:

Define a subscription mechanism to notify multiple objects about events that happen to the object they're observing. The subject maintains a list of its dependents (observers) and notifies them automatically when its state changes.

Key components:
- **Subject**: Maintains a list of observers and provides methods to add/remove them
- **Observer**: Defines an interface for objects that should be notified of changes
- **ConcreteSubject**: Implements the Subject interface and sends notifications to observers when its state changes
- **ConcreteObserver**: Implements the Observer interface to respond to updates from the subject

### Diagram:

```mermaid
classDiagram
    class Subject {
        <<interface>>
        +attach(observer)
        +detach(observer)
        +notify()
    }
    class Observer {
        <<interface>>
        +update(subject)
    }
    class ConcreteSubject {
        -state
        -observers: list
        +attach(observer)
        +detach(observer)
        +notify()
        +setState(newState)
        +getState()
    }
    class ConcreteObserver {
        -state
        +update(subject)
    }
    
    Subject <|-- ConcreteSubject
    Observer <|-- ConcreteObserver
    ConcreteSubject o-- Observer : notifies >
```

In [7]:
from abc import ABC, abstractmethod
from typing import List


# Observer interface
class Observer(ABC):
    @abstractmethod
    def update(self, subject) -> None:
        pass


# Subject interface
class Subject(ABC):
    @abstractmethod
    def attach(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def detach(self, observer: Observer) -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass

In [8]:
# Concrete Subject - Weather Station
class WeatherStation(Subject):
    def __init__(self):
        self._observers: List[Observer] = []
        self._temperature = 0
        self._humidity = 0
        self._pressure = 0

    def attach(self, observer: Observer) -> None:
        if observer not in self._observers:
            self._observers.append(observer)
            print(f"Subject: Attached an observer.")

    def detach(self, observer: Observer) -> None:
        self._observers.remove(observer)
        print(f"Subject: Detached an observer.")

    def notify(self) -> None:
        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer.update(self)

    def set_measurements(self, temperature, humidity, pressure):
        print("\nWeather Station: Setting new measurements...")
        self._temperature = temperature
        self._humidity = humidity
        self._pressure = pressure
        self.notify()

    @property
    def temperature(self):
        return self._temperature

    @property
    def humidity(self):
        return self._humidity

    @property
    def pressure(self):
        return self._pressure

In [9]:
# Concrete Observers
class CurrentConditionsDisplay(Observer):
    def update(self, weather_station):
        print(f"Current Conditions: {weather_station.temperature}°C, {weather_station.humidity}% humidity")


class StatisticsDisplay(Observer):
    def __init__(self):
        self.temperatures = []

    def update(self, weather_station):
        self.temperatures.append(weather_station.temperature)
        avg = sum(self.temperatures) / len(self.temperatures)
        print(
            f"Statistics: Avg temperature = {avg:.1f}°C, "
            f"Max = {max(self.temperatures)}°C, "
            f"Min = {min(self.temperatures)}°C"
        )


class ForecastDisplay(Observer):
    def __init__(self):
        self.last_pressure = 0

    def update(self, weather_station):
        if self.last_pressure == 0:
            print("Forecast: Waiting for more data...")
        elif weather_station.pressure > self.last_pressure:
            print("Forecast: Improving weather on the way!")
        elif weather_station.pressure < self.last_pressure:
            print("Forecast: Watch out for cooler, rainy weather")
        else:
            print("Forecast: More of the same")

        self.last_pressure = weather_station.pressure

In [10]:
# Client code
# Create the subject
weather_station = WeatherStation()

# Create observers
current_display = CurrentConditionsDisplay()
statistics_display = StatisticsDisplay()
forecast_display = ForecastDisplay()

# Register observers
weather_station.attach(current_display)
weather_station.attach(statistics_display)
weather_station.attach(forecast_display)

# Simulate weather changes
weather_station.set_measurements(25, 65, 1013)
weather_station.set_measurements(26, 70, 1014)
weather_station.set_measurements(24, 90, 1010)

# Unregister an observer
weather_station.detach(forecast_display)

# One more measurement
weather_station.set_measurements(22, 80, 1012)

Subject: Attached an observer.
Subject: Attached an observer.
Subject: Attached an observer.

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 25°C, 65% humidity
Statistics: Avg temperature = 25.0°C, Max = 25°C, Min = 25°C
Forecast: Waiting for more data...

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 26°C, 70% humidity
Statistics: Avg temperature = 25.5°C, Max = 26°C, Min = 25°C
Forecast: Improving weather on the way!

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 24°C, 90% humidity
Statistics: Avg temperature = 25.0°C, Max = 26°C, Min = 24°C
Forecast: Watch out for cooler, rainy weather
Subject: Detached an observer.

Weather Station: Setting new measurements...
Subject: Notifying observers...
Current Conditions: 22°C, 80% humidity
Statistics: Avg temperature = 24.2°C, Max = 26°C, Min = 22°C


### Alternative Python Implementation using built-in mechanisms:

In [11]:
# Using Python's built-in Observer pattern capabilities with callbacks
class EventPublisher:
    def __init__(self):
        self._subscribers = {}

    def subscribe(self, event_type, callback):
        if event_type not in self._subscribers:
            self._subscribers[event_type] = []
        self._subscribers[event_type].append(callback)
        return callback  # Return for easy unsubscribing

    def unsubscribe(self, event_type, callback):
        if event_type in self._subscribers and callback in self._subscribers[event_type]:
            self._subscribers[event_type].remove(callback)

    def publish(self, event_type, *args, **kwargs):
        if event_type in self._subscribers:
            for callback in self._subscribers[event_type]:
                callback(*args, **kwargs)


# Example usage
if __name__ == "__main__":
    # Create publisher
    weather_publisher = EventPublisher()

    # Define callbacks
    def display_temperature(temp):
        print(f"Current temperature: {temp}°C")

    def alert_extreme_weather(temp):
        if temp > 35:
            print("ALERT: Very hot weather!")
        elif temp < 0:
            print("ALERT: Freezing conditions!")

    # Subscribe to events
    weather_publisher.subscribe("temperature_change", display_temperature)
    weather_publisher.subscribe("temperature_change", alert_extreme_weather)

    # Publish events
    print("\nSimulating weather changes:")
    weather_publisher.publish("temperature_change", 28)
    weather_publisher.publish("temperature_change", 36)

    # Unsubscribe from events
    weather_publisher.unsubscribe("temperature_change", alert_extreme_weather)
    print("\nAfter unsubscribing alert system:")
    weather_publisher.publish("temperature_change", 38)


Simulating weather changes:
Current temperature: 28°C
Current temperature: 36°C
ALERT: Very hot weather!

After unsubscribing alert system:
Current temperature: 38°C


### Real-world analogies:

1. **News Subscription Service**:
   - Subject: News publisher or newspaper company
   - Observers: Subscribers who receive newspapers or news updates
   - Subscription/unsubscription: Signing up for or canceling a subscription
   - Notification: Delivery of newspapers or news alerts

2. **Social Media Notifications**:
   - Subject: Social media platform or specific content creator
   - Observers: Followers who have enabled notifications
   - Subscription/unsubscription: Following/unfollowing or toggling notification settings
   - Notification: Push notifications when new content is posted

### When to use:

- When changes to the state of one object require changing other objects, and the number of objects that need to change is unknown or dynamic
- When an object should be able to notify other objects without making assumptions about what those objects are
- When you need a loosely coupled design between interacting objects 
- For implementing distributed event handling systems
- In user interface development for separating views from models

### Python-specific implementation notes:

- Python offers several built-in mechanisms to implement observer-like behavior:
  - **Callback functions and lambdas** provide lightweight observer implementations
  - **Event libraries** like PyPubSub provide full pub/sub functionality
  - **Descriptors and properties** can notify on attribute changes
  - The **asyncio** library provides event-driven programming capabilities
- Python's duck typing allows for more flexible observer implementations than in statically typed languages
- The `@property` decorator combined with setters can implement simple notification mechanisms

### Related patterns:

- **Mediator**: While Observer distributes communication by introducing publisher and subscriber objects, Mediator centralizes communication between objects
- **Singleton**: The Subject is often implemented as a Singleton when exactly one instance is needed
- **Command**: Commands can be used to implement the publish-subscribe relationship
- **Strategy**: Observers often use different strategies to process notifications from the subject

### Benefits

* **Loose Coupling**: Subjects don't need to know anything about their observers
* **Open/Closed Principle**: You can add new observer classes without changing the subject
* **Dynamic Relationships**: You can establish relationships between objects at runtime
* **Broadcast Communication**: A subject can notify all interested objects automatically
* **Event-Driven Architecture**: Supports building event-driven systems and reactive programming